In [6]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator

def create_bv_oracle(secret_string):
    """
    f(x) = x·s (mod 2) 구현

    예 : secret_string = "101"
    -> 0번의 큐비트와 2번 큐비트에만 CNOT
    """

    n = len(secret_string)
    oracle = QuantumCircuit(n+1, name='BV_Oracle')

    # "101" -> reversed = "101"
    # i=0, bit='1' -> cx(0,n) 추가
    # i=1. bit='0' -> 건너뜀
    # i = 2, bit='1' -> cx(2,n)추가
    for i, bit in enumerate(reversed(secret_string)):
        if bit == '1':
            oracle.cx(i, n)
    return oracle

In [13]:
def bernstein_vazirani_circuit(secret_string):
    n = len(secret_string)

    #레지스터
    input_qubits = QuantumRegister(n, 'input')
    output_qubit = QuantumRegister(1, 'output')
    classical_bits = ClassicalRegister(n, 'result')

    qc = QuantumCircuit(input_qubits, output_qubit, classical_bits)

    # |1> 초기화
    qc.x(output_qubit[0])

    # hadamard로 중첩 생성
    qc.h(input_qubits) # |0> -> |+>
    qc.h(output_qubit) # |1> -> |->

    qc.barrier()

    #오라클(위상 인코딩)
    oracle = create_bv_oracle(secret_string)
    qc.append(oracle, input_qubits[:] + output_qubit[:])

    qc.barrier()

    # 다시 hadamard(간섭)
    qc.h(input_qubits)
    qc.barrier()

    # 측정
    qc.measure(input_qubits, classical_bits)

    return qc

In [17]:
# secret string 설정
SECRET = '101'

#회로 생성
qc = bernstein_vazirani_circuit(SECRET)

# 시뮬레이터 실행
simulator = AerSimulator()
compiled = transpile(qc, simulator)
job = simulator.run(compiled, shots=1024)
result = job.result()
counts = result.get_counts()

print(f"Secret: \"{SECRET}\"")
print(f"결과: {counts}")
print(f"찾은 string: \"{max(counts, key=counts.get)}\"")

Secret: "101"
결과: {'101': 1024}
찾은 string: "101"


In [18]:
#양자컴

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    token='')

backend = service.backends(simulator=False, operational=True)[0]
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pm.run(qc)

sampler = Sampler(backend)
job = sampler.run([isa_circuit], shots=1024)
result = job.result()

# 결과 처리
counts_hw = result[0].data.result.get_counts()
counts_hw = {k[::-1]: v for k, v in counts_hw.items()}

print(f"실제 양자컴: {counts_hw}")

qiskit_runtime_service.__init__:WARNING:2025-12-21 12:16:50,993: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: dotnanimoe. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-21 12:16:50,994: Loading instance: dotnanimoe, plan: open


실제 양자컴: {'101': 955, '100': 19, '000': 5, '001': 35, '011': 1, '111': 9}
